# DIA4. Carga de datos y estrategia de destino

## Ejercicio: Implementar diferentes estrategias de carga
---

### 1.- Preparar datos de ejemplo

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

# Generar datos de ventas
np.random.seed(42)
ventas = pd.DataFrame({
    'venta_id': range(1, 1001),
    'cliente_id': np.random.randint(1, 101, 1000),
    'producto_id': np.random.randint(1, 51, 1000),
    'cantidad': np.random.randint(1, 11, 1000),
    'precio_unitario': np.round(np.random.uniform(10, 500, 1000), 2),
    'fecha_venta': pd.date_range('2024-01-01', periods=1000, freq='1H'),
    'updated_at': datetime.now()
})

ventas['total'] = ventas['cantidad'] * ventas['precio_unitario']

print(f"Generados {len(ventas)} registros de ventas")
print(ventas.head())

Generados 1000 registros de ventas
   venta_id  cliente_id  producto_id  cantidad  precio_unitario  \
0         1          52           34         4           405.14   
1         2          93           47        10           235.03   
2         3          15            8         6            35.46   
3         4          72           40         7           395.28   
4         5          61           49         2           108.67   

          fecha_venta                 updated_at    total  
0 2024-01-01 00:00:00 2026-02-05 23:27:00.363165  1620.56  
1 2024-01-01 01:00:00 2026-02-05 23:27:00.363165  2350.30  
2 2024-01-01 02:00:00 2026-02-05 23:27:00.363165   212.76  
3 2024-01-01 03:00:00 2026-02-05 23:27:00.363165  2766.96  
4 2024-01-01 04:00:00 2026-02-05 23:27:00.363165   217.34  


/tmp/ipython-input-1391396460.py:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'fecha_venta': pd.date_range('2024-01-01', periods=1000, freq='1H'),


### 2.- Carga completa (full load)

In [5]:
import sqlite3

def carga_completa_sqlite(df, tabla):
    conn = sqlite3.connect(':memory:')

    # Crear tabla
    conn.execute(f'''
        CREATE TABLE {tabla} (
            venta_id INTEGER PRIMARY KEY,
            cliente_id INTEGER,
            producto_id INTEGER,
            cantidad INTEGER,
            precio_unitario REAL,
            total REAL,
            fecha_venta TEXT,
            updated_at TEXT
        )
    ''')

    # Insertar datos
    df.to_sql(tabla, conn, if_exists='replace', index=False)

    # Verificar
    cursor = conn.execute(f"SELECT COUNT(*) FROM {tabla}")
    count = cursor.fetchone()[0]

    conn.close()
    return count

registros_cargados = carga_completa_sqlite(ventas, 'ventas_completas')
print(f"Carga completa: {registros_cargados} registros")

Carga completa: 1000 registros


### 3.- Carga incremental simulada

In [6]:
def carga_incremental(df, archivo_parquet, ultimo_id=0):
    # Simular carga incremental: solo registros nuevos
    nuevos_registros = df[df['venta_id'] > ultimo_id]

    if len(nuevos_registros) > 0:
        try:
            # En producción, leer archivo existente y append
            nuevos_registros.to_parquet(
                archivo_parquet,
                engine='pyarrow',
                index=False
            )
            print(f"Carga incremental: {len(nuevos_registros)} nuevos registros")
            return len(nuevos_registros)
        except Exception as e:
            print(f"Error en carga incremental: {e}")
            return 0
    else:
        print("No hay nuevos registros para cargar")
        return 0

nuevos_cargados = carga_incremental(ventas, 'ventas_incremental.parquet', ultimo_id=500)
print(f"Registros nuevos agregados: {nuevos_cargados}")

Carga incremental: 500 nuevos registros
Registros nuevos agregados: 500
